## 12. 네거티브 샘플링(Negative Sampling) Word2Vec
- 네거티브 샘플링은 Word2Vec이 학습 과정에서 전체 단어 집합이 아닌, 일부 단어 집합에만 집중할 수 있도록 하는 방법
- Word2Vec은 역전파 과정에서 모든 단어의 임베딩 벡터값의 업데이트를 수행
    - EX) 만약 현재 집중하고 있는 중심 단어와 주변 단어가 '강아지'와 '고양이', '귀여운'과 같은 단어라면, 별 연관 관계가 없는 '돈가스'나 '컴퓨터'와 같은 수많은 단어의 임베딩 벡터값까지 업데이트하는 것은 비효율적
- SGNS(Skip-Gram with Negative Sampling)는 다음과 같이 중심 단어와 주변 단어가 모두 입력이 되고, 이 두 단어가 실제로 윈도우 크기 내에 존재하는 이웃 관계인지 그 확률을 예측

## 12.1 SGNS
- Skip-gram은 기본적으로 중심 단어를 입력, 주변 단어를 레이블로 진행
- SGNS는 중심단어(입력1), 주변단어(주변2)로 두고 실제로 윈도우 크기 내 이웃관계엿다면 레이블이 1
- 단어 집합에서 랜덤으로 선택한 단어들을 입력2로 하고 레이블을 0으로 함
![negative_sampling](img/negative_sampling.png)

## 12.2 20뉴스그룹 데이터 전처리하기

In [2]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from sklearn.datasets import fetch_20newsgroups
from tensorflow.keras.preprocessing.text import Tokenizer

# LSA 챕터에서 사용했던 20뉴스그룹 데이터를 재사용
dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data
print('총 샘플 수 :',len(documents))

총 샘플 수 : 11314


In [3]:
# 불필요한 토큰을 제거하고, 소문자화를 통해 정규화를 진행
news_df = pd.DataFrame({'document':documents})
# 특수 문자 제거
news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]", " ")
# 길이가 3이하인 단어는 제거 (길이가 짧은 단어 제거)
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
# 전체 단어에 대한 소문자 변환
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: x.lower())

# null 값 확인
news_df.isnull().values.any()
news_df.replace("", float("NaN"), inplace=True)
news_df.isnull().values.any()

news_df.dropna(inplace=True)
print('총 샘플 수 :',len(news_df))

C:\Users\Black\AppData\Local\Temp/ipykernel_8656/3162781224.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]", " ")


총 샘플 수 : 10995


In [7]:
# 불용어를 제거
stop_words = stopwords.words('english')
tokenized_doc = news_df['clean_doc'].apply(lambda x: x.split())
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])
tokenized_doc = tokenized_doc.to_list()

# 단어가 1개 이하인 샘플의 인덱스를 찾아서 저장하고, 해당 샘플들은 제거.
drop_train = [index for index, sentence in enumerate(tokenized_doc) if len(sentence) <= 1]
tokenized_doc = np.delete(tokenized_doc, drop_train, axis=0)
print('총 샘플 수 :',len(tokenized_doc))

# 단어 집합 생성 및 Toeknizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(tokenized_doc)

word2idx = tokenizer.word_index
idx2word = {v:k for k, v in word2idx.items()}
encoded = tokenizer.texts_to_sequences(tokenized_doc)
print(encoded[:2])

vocab_size = len(word2idx) + 1 
print('단어 집합의 크기 :', vocab_size)

총 샘플 수 : 10940
[[9, 59, 603, 207, 3278, 1495, 474, 702, 9470, 13686, 5533, 15227, 702, 442, 702, 70, 1148, 1095, 1036, 20294, 984, 705, 4294, 702, 217, 207, 1979, 15228, 13686, 4865, 4520, 87, 1530, 6, 52, 149, 581, 661, 4406, 4988, 4866, 1920, 755, 10668, 1102, 7837, 442, 957, 10669, 634, 51, 228, 2669, 4989, 178, 66, 222, 4521, 6066, 68, 4295], [1026, 532, 2, 60, 98, 582, 107, 800, 23, 79, 4522, 333, 7838, 864, 421, 3825, 458, 6488, 458, 2700, 4730, 333, 23, 9, 4731, 7262, 186, 310, 146, 170, 642, 1260, 107, 33568, 13, 985, 33569, 33570, 9471, 11491]]
단어 집합의 크기 : 64277


## 12.3 네거티브 샘플링을 통한 데이터셋 구성하기
- 네거티브 샘플링을 통한 데이터셋을 구성
- 네거티브 샘플링을 위해서 케라스에서 제공하는 전처리 도구인 skipgrams를 사용
- 상위 10개의 뉴스그룹 샘플에 대해서만 수행

In [10]:
from tensorflow.keras.preprocessing.sequence import skipgrams

# 네거티브 샘플링 - 10개만
skip_grams = [skipgrams(sample, vocabulary_size=vocab_size, window_size=10) for sample in encoded[:10]]

# 첫번째 샘플인 skip_grams[0] 내 skipgrams로 형성된 데이터셋 확인
pairs, labels = skip_grams[0][0], skip_grams[0][1]
for i in range(5):
    print("({:s} ({:d}), {:s} ({:d})) -> {:d}".format(
          idx2word[pairs[i][0]], pairs[i][0], 
          idx2word[pairs[i][1]], pairs[i][1], 
          labels[i]))
    
# 첫번째 뉴스그룹 샘플에 대해서 생긴 pairs와 labels의 개수
print(len(pairs))
print(len(labels))

# 네거티브 샘플링 - 전체
skip_grams = [skipgrams(sample, vocabulary_size=vocab_size, window_size=10) for sample in encoded]


(shame (4988), conferences (5928)) -> 0
(existance (4865), report (581)) -> 1
(commited (7837), nazism (8057)) -> 0
(least (87), souped (49270)) -> 0
(holocaust (2669), unfortunate (4295)) -> 1
2220
2220


## 12.4 Skip-Gram with Negative Sampling(SGNS) 구현
- 각 임베딩 테이블은 중심 단어와 주변 단어 각각을 위한 임베딩 테이블
- 각 단어는 임베딩 테이블을 거쳐서 내적을 수행하고, 내적의 결과는 1 또는 0을 예측하기 위해서 시그모이드 함수를 활성화 함수로 거쳐 최종 예측값을 얻음

In [11]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, Reshape, Activation, Input
from tensorflow.keras.layers import Dot
from tensorflow.keras.utils import plot_model
from IPython.display import SVG

# 임베딩 벡터 차원
embed_size = 100

# 중심 단어를 위한 임베딩 테이블
w_inputs = Input(shape=(1, ), dtype='int32')
word_embedding = Embedding(vocab_size, embed_size)(w_inputs)

# 주변 단어를 위한 임베딩 테이블
c_inputs = Input(shape=(1, ), dtype='int32')
context_embedding  = Embedding(vocab_size, embed_size)(c_inputs)


dot_product = Dot(axes=2)([word_embedding, context_embedding])
dot_product = Reshape((1,), input_shape=(1, 1))(dot_product)
output = Activation('sigmoid')(dot_product)

model = Model(inputs=[w_inputs, c_inputs], outputs=output)
model.summary()
model.compile(loss='binary_crossentropy', optimizer='adam')
plot_model(model, to_file='model3.png', show_shapes=True, show_layer_names=True, rankdir='TB')

for epoch in range(1, 6):
    loss = 0
    for _, elem in enumerate(skip_grams):
        first_elem = np.array(list(zip(*elem[0]))[0], dtype='int32')
        second_elem = np.array(list(zip(*elem[0]))[1], dtype='int32')
        labels = np.array(elem[1], dtype='int32')
        X = [first_elem, second_elem]
        Y = labels
        loss += model.train_on_batch(X,Y)  
    print('Epoch :',epoch, 'Loss :',loss)
    
    

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 1, 100)       6427700     input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1, 100)       6427700     input_2[0][0]                    
______________________________________________________________________________________________

In [12]:
import gensim

# 학습된 임베딩 벡터들 저장
f = open('vectors.txt' ,'w')
f.write('{} {}\n'.format(vocab_size-1, embed_size))
vectors = model.get_weights()[0]
for word, i in tokenizer.word_index.items():
    f.write('{} {}\n'.format(word, ' '.join(map(str, list(vectors[i, :])))))
f.close()

# gensim의 .models.KeyedVectors.load_word2vec_format로 로드하여 단어 간 유사도 구함
w2v = gensim.models.KeyedVectors.load_word2vec_format('./vectors.txt', binary=False)
print(w2v.most_similar(positive=['soldiers']))
print(w2v.most_similar(positive=['doctor']))

[('terrorist', 0.8891729116439819), ('wounded', 0.8860478401184082), ('bosnia', 0.8706362843513489), ('massacred', 0.8636177182197571), ('murdered', 0.8593177199363708), ('slaughtered', 0.8588657379150391), ('nazis', 0.8575608730316162), ('inhabitants', 0.8543673753738403), ('syrian', 0.8535692691802979), ('kuwait', 0.8510521650314331)]
[('doctors', 0.6503483653068542), ('medicine', 0.6230402588844299), ('wounds', 0.6080016493797302), ('diagnosed', 0.5967128872871399), ('chronic', 0.5953965783119202), ('obsession', 0.585250735282898), ('oral', 0.584855318069458), ('suffered', 0.579618513584137), ('physician', 0.5793462991714478), ('illness', 0.5725647807121277)]
